In [6]:
from pathlib import Path
import pandas as pd
import numpy as np
import sncosmo
from astropy.cosmology import FlatLambdaCDM

project_root = Path.cwd().parent
print(f"Project root: {project_root}")

Project root: /Users/david/Code/msc


In [8]:
# User input: run folder name 
folder_name = input("Enter the run folder name: ").strip()
run_folder = project_root / "runs" / folder_name

params_path = run_folder / "sncosmo_fits.csv"

df = pd.read_csv(params_path)
print(f"Loaded {len(df)} object(s) from {params_path}")
df.head()

# c : colour paramter, + means redder
# x1 : stretch parameter, + means broader lightcurve
# redshift : calculated from sncosmo fit
# t0 : time of maximum brightness in B-band
# x0 : overall flux scaling parameter
# x1 : stretch parameter, + means broader lightcurve

Enter the run folder name: test4
Loaded 423 object(s) from /Users/david/Code/msc/runs/test4/sncosmo_fits.csv


,ztf_id,redshift,ncall,ndof,chisq,t0,x0,x1,c
0,ZTF17aabtvsy,0.069222,179,19,59.986348,59863.790783,0.000851,7.054899,0.162527
1,ZTF17aabvong,0.034000,92,27,274.722693,60607.461064,0.005095,0.122704,-0.036969
2,ZTF17aacldgo,0.072000,128,13,56.212345,59898.042470,0.000568,-7.269984,0.004402
3,ZTF17aadlxmv,0.062000,102,24,61.273738,58878.493399,0.001053,0.714857,0.123591
4,ZTF18aaaonon,0.071419,131,9,108.298523,59707.375818,0.000690,-2.306936,-0.075222


In [9]:
# Nuisance parameters
alpha = 0.14
beta = 3.1
M = -19.36
cosmo = FlatLambdaCDM(H0=70.0, Om0=0.3)

# build model once (faster), then set params each loop
model = sncosmo.Model(source="salt3")

ztf_cleansed_path = project_root / "ztf_cleansed.csv"
ztf_df = pd.read_csv(ztf_cleansed_path)
processed_rows = []

for idx, row in df.iterrows():
    ztf_id = row['ztf_id']

    try:
        z = ztf_df.loc[ztf_df['ZTFID'] == ztf_id, 'host_redshift'].iloc[0]
    except IndexError:
        z = row['redshift']

    if pd.isna(z) or z == '' or (isinstance(z, str) and not str(z).strip()):
        z = row['redshift']
    z = float(z)

    ncall = row['ncall']
    ndof = row['ndof']
    chisq = row['chisq']
    t0 = float(row['t0'])
    x0 = float(row['x0'])
    x1 = float(row['x1'])
    c  = float(row['c'])

    # SALT3 safe apparent_mag_B
    model.set(z=z, t0=t0, x0=x0, x1=x1, c=c)
    apparent_mag_B = model.bandmag("bessellb", "ab", model.get("t0"))   # apparent magnitude from rest-frame B
    distance_modulus_u_obs = apparent_mag_B - M # + alpha * x1 - beta * c  # distance modulus after correction
    mu_theoretical  = cosmo.distmod(z).value # distance modulus from cosmology
    resid  = distance_modulus_u_obs - mu_theoretical

    processed_rows.append({
        'ztf_id': ztf_id,
        'host_redshift': z,
        'ncall': ncall,
        'ndof': ndof,
        'chisq': chisq,
        't0': t0,
        'x0': x0,
        'x1': x1,
        'c': c,
        'apparent_mag_B': apparent_mag_B,
        'distance_modulus_u_obs': distance_modulus_u_obs,
        'mu_th': mu_theoretical,
        'resid': resid,
    })

    print(f"{ztf_id}: z={z:.5f}, apparent_mag_B={apparent_mag_B:.3f}, distance_modulus_u_obs={distance_modulus_u_obs:.3f}, mu_theoretical={mu_theoretical:.3f}, resid={resid:.3f}")

output_df = pd.DataFrame(processed_rows)
output_path = run_folder / "distance_process.csv"
output_df.to_csv(output_path, index=False)
print(f"\nWrote processed results to {output_path}")


ZTF17aabtvsy: z=0.06922, apparent_mag_B=18.184, distance_modulus_u_obs=37.544, mu_theoretical=37.471, resid=0.074
ZTF17aabvong: z=0.03400, apparent_mag_B=16.239, distance_modulus_u_obs=35.599, mu_theoretical=35.872, resid=-0.273
ZTF17aacldgo: z=0.07200, apparent_mag_B=18.761, distance_modulus_u_obs=38.121, mu_theoretical=37.560, resid=0.561
ZTF17aadlxmv: z=0.06200, apparent_mag_B=18.005, distance_modulus_u_obs=37.365, mu_theoretical=37.220, resid=0.144
ZTF18aaaonon: z=0.07142, apparent_mag_B=18.458, distance_modulus_u_obs=37.818, mu_theoretical=37.542, resid=0.276
ZTF18aaaooqj: z=0.05066, apparent_mag_B=18.606, distance_modulus_u_obs=37.966, mu_theoretical=36.764, resid=1.202
ZTF18aaaqexr: z=0.07020, apparent_mag_B=18.648, distance_modulus_u_obs=38.008, mu_theoretical=37.503, resid=0.505
ZTF18aadlaxo: z=0.05295, apparent_mag_B=18.102, distance_modulus_u_obs=37.462, mu_theoretical=36.864, resid=0.598
ZTF18aadxnul: z=0.06400, apparent_mag_B=18.166, distance_modulus_u_obs=37.526, mu_theor